In [1]:
!nvidia-smi

Sat Nov 16 23:36:19 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   45C    P0    55W / 300W |    849MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
%load_ext autoreload
%autoreload 2

import skfmm
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from obspy import UTCDateTime as UT

from harpa.NF_utils import *

In [3]:
config={}
config['x(km)']=[0,100]
config['y(km)']=[0,100]
config['z(km)']=[0,100]
config["center"] = ((config['x(km)'][0]+config['x(km)'][1])/2, (config['y(km)'][0]+config['y(km)'][1])/2)


dx=4
config['dx(km)']=dx   
grid_size_x= int((config['x(km)'][1]-config['x(km)'][0])/dx+1)
grid_size_y= int((config['y(km)'][1]-config['y(km)'][0])/dx+1)
grid_size_z= int((config['z(km)'][1]-config['z(km)'][0])/dx+1)
x = np.linspace(config['x(km)'][0],config['x(km)'][1],grid_size_x)
y = np.linspace(config['y(km)'][0],config['y(km)'][1],grid_size_y)
z = np.linspace(config['z(km)'][0],config['z(km)'][1],grid_size_z)
X, Y, Z = np.meshgrid(x,y,z)
X=X.transpose(1,0,2)
Y=Y.transpose(1,0,2)
Z=Z.transpose(1,0,2)
X0=X.copy()
Y0=Y.copy()
Z0=Z.copy()


In [4]:
# Super simple model with only 4 paramters with 1 or 2 Gaussian pertubation
n_train_wavespeed=100
v_background=6

V_list=[]
p_list=[]
np.random.seed(0)
for i in range(n_train_wavespeed):
    V=np.ones_like(Z)*2
    p0=np.random.rand()*0.9+0.1
    p1=np.random.rand()*0.9+0.1
    V=V+add_gaussian_pertubation(0.5,0.5,p0,p0*0.5,p0*0.5,p1*0.5,30*p1,grid_size_x)
    if np.random.rand()<-1:
        p2=np.random.rand()*0.9+0.1
        p3=np.random.rand()*0.9+0.1
        V=V+add_gaussian_pertubation(0.5,0.5,p2,p2*0.5,p2*0.5,p3*0.5,30*p3,grid_size_x)
    else:
        p2=0
        p3=0
    #p=[p0,p1,p2,p3]
    p=[p1]
    V[V<1]=1
    V[V>18]=18
    V=V/V.mean()*v_background
    V_list.append(V)
    p_list.append(p)

In [5]:
WaveSpeedData=WaveSpeedDataset(V_list,p_list)

In [7]:
from harpa.model import Autoencoder
import torch
device='cuda:0'
L=2
model_autoencoder=Autoencoder(capacity=64,latent_dims=L).to(device)
ATE_train_loader = torch.utils.data.DataLoader(WaveSpeedData, batch_size=128, shuffle=True)
optimizer = torch.optim.Adam(params=model_autoencoder.parameters(), lr=1e-3)

In [8]:
# use more than 1 GPUs
# if torch.cuda.device_count() > 1:
#     print("Using", torch.cuda.device_count(), "GPUs!")
#     model_autoencoder = torch.nn.DataParallel(model_autoencoder)
# model_autoencoder.to('cuda')
# device='cuda'

In [9]:
model_autoencoder.train()
train_loss_avg = []
num_epochs=1000000
print('Training ...')
import torch.nn.functional as F
for epoch in range(num_epochs):
    train_loss_avg.append(0)
    num_batches = 0
    #for data in tqdm(ATE_train_loader):
    for data in ATE_train_loader:
        image_batch = data['image'].to(device)
        image_batch=image_batch.unsqueeze(1)
        image_batch_recon,_ = model_autoencoder(image_batch)
        loss = F.mse_loss(image_batch_recon, image_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss_avg[-1] += loss.item()
        num_batches += 1
    train_loss_avg[-1] /= num_batches
    if train_loss_avg[-1]<1e-2:
        break
    if epoch%50==0:
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch+1, num_epochs, train_loss_avg[-1]))
        torch.save(model_autoencoder.state_dict(), 'checkpoint/model_autoencoder.pt')

Training ...
Epoch [1 / 1000000] average reconstruction error: 54.174355
Epoch [51 / 1000000] average reconstruction error: 1.272551
Epoch [101 / 1000000] average reconstruction error: 0.593939
Epoch [151 / 1000000] average reconstruction error: 0.325769
Epoch [201 / 1000000] average reconstruction error: 0.217531
Epoch [251 / 1000000] average reconstruction error: 0.120475
Epoch [301 / 1000000] average reconstruction error: 0.121518
Epoch [351 / 1000000] average reconstruction error: 0.086283
Epoch [401 / 1000000] average reconstruction error: 0.081308
Epoch [451 / 1000000] average reconstruction error: 0.085758
Epoch [501 / 1000000] average reconstruction error: 0.050305
Epoch [551 / 1000000] average reconstruction error: 0.035742
Epoch [601 / 1000000] average reconstruction error: 0.031590
Epoch [651 / 1000000] average reconstruction error: 0.032460
Epoch [701 / 1000000] average reconstruction error: 0.026743
Epoch [751 / 1000000] average reconstruction error: 0.023582
Epoch [801 / 

In [10]:
n_station=20
n_source=8
frequence  = 3
station_df=gen_station(n_station,config)
#travel_time_list=get_traveling_time_multi(V_list,station_df,config,dx=dx)
travel_time_list=get_traveling_time_multi_mp(V_list,station_df,config,dx=dx)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:01<00:00, 1082.49it/s]


In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
ttdata=TravelTimeDataset_NF(travel_time_list,WaveSpeedData,p_list,model_autoencoder,device,config,beta=1)

z, [[0.6732348  0.38452196]
 [0.73242456 0.33168945]
 [0.6800483  0.4523244 ]
 [0.7180063  0.48992893]
 [0.6066132  0.42000672]]
(26, 26, 26) (26, 26, 26) (26, 26, 26)
(100, 20, 26, 26, 26)


In [13]:
TT1=ttdata.travel_time_list[0,0]
TT1.min(),TT1.max()

(2.75e-05, 23.47)

In [14]:
import torch
from harpa.model import Siren
device='cuda:0'
L=2
#batch_size_siren=10000

#batch_size_siren=20
first_omega_0=30
hidden_omega_0=30
model_traveltime= Siren(in_features=3+L, out_features=len(station_df), hidden_features=128, 
                        hidden_layers=3, outermost_linear=True,first_omega_0=first_omega_0, hidden_omega_0=hidden_omega_0).to(device)
optimizer = torch.optim.Adam(model_traveltime.parameters(), lr=0.0001)
loss_function=torch.nn.MSELoss(reduction='mean')
#train_loader = torch.utils.data.DataLoader(ttdata, batch_size=batch_size_siren, shuffle=True,num_workers=8)

#train_loader = torch.utils.data.DataLoader(ttdata, batch_size=batch_size_siren, shuffle=True)

In [15]:
# use more than 1 GPUs
# if torch.cuda.device_count() > 1:
#     print("Using", torch.cuda.device_count(), "GPUs!")
#     model_traveltime = torch.nn.DataParallel(model_traveltime,device_ids=[0,1,2,3])

In [16]:
batch_size_siren=1000
train_loader = torch.utils.data.DataLoader(ttdata, batch_size=batch_size_siren, shuffle=True)
device='cuda:0'
_=model_traveltime.to(device)

In [17]:
for epoch in range(100):
    model_traveltime.train()
    loss_batch = 0
    for data in tqdm(train_loader):
        X = data['in'].float().to(device)
        Y = data['out'].float().to(device)
        Ypred,_= model_traveltime(X)
        loss = loss_function(Y, Ypred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_batch += loss.item()

    if epoch%1==0:
        loss_epoch = loss_batch/len(train_loader)
        print(f'Epoch: {epoch:03d}, Train Loss: {loss_epoch:.2E}')
        torch.save(model_traveltime.state_dict(), 'checkpoint/model_traveltime.pt')
    if loss_epoch<1e-2:
        break

  0%|          | 0/1758 [00:00<?, ?it/s]

100%|██████████| 1758/1758 [00:32<00:00, 54.18it/s]


Epoch: 000, Train Loss: 1.19E+02


 69%|██████▊   | 1205/1758 [00:22<00:10, 53.19it/s]


KeyboardInterrupt: 

# Generate test data

In [18]:
frequence=3

In [34]:
#n_source=8
source_loc=[]
source_loc_lindex=[]
source_time=[]
Tmax=60/frequence*n_source
Tlist=np.linspace(0,Tmax,n_source)
#Tlist=np.random.rand(n_source)*Tmax
for n in range(n_source):
    #idxs=[np.random.randint(len(x)),np.random.randint(len(y)),np.random.randint(len(z))]
    idxs=[np.random.randint(1,len(x)-1),np.random.randint(1,len(y)-1),np.random.randint(5,len(z)-1)] #avoid boundary
    source_loc_lindex.append(idxs)
    source_loc.append([x[idxs[0]],y[idxs[1]],z[idxs[2]]])
    #source_time.append(np.random.rand()*60/frequence*8)
    source_time.append(Tlist[n])
    
catalog_df=[]
for source_index in range(n_source):
    loc=source_loc[source_index]
    catalog_df.append({
        "event_index": source_index,
        "time":  np.datetime64(int(1000*source_time[source_index]), 'ms'),#UT(np.datetime64(int(1000*source_time[source_index]), 'ms').astype('datetime64[ms]').astype(int) / 1000.0),
        "x(km)": loc[0],
        "y(km)": loc[1],
        "z(km)": loc[2],
    })
catalog_df=pd.DataFrame(catalog_df)
catalog_df['time'] = catalog_df['time'].apply(lambda x: UT(x))

In [20]:
V=np.ones_like(Z)*2
p0=np.random.rand()*0.9+0.1
p1=np.random.rand()*0.9+0.1
V=V+add_gaussian_pertubation(0.5,0.5,p0,p0*0.5,p0*0.5,p1*0.5,30*p1,grid_size_x)
if np.random.rand()<-1:
    p2=np.random.rand()*0.9+0.1
    p3=np.random.rand()*0.9+0.1
    V=V+add_gaussian_pertubation(0.5,0.5,p2,p2*0.5,p2*0.5,p3*0.5,30*p3,grid_size_x)
V_test=V

In [22]:
# Xin=torch.tensor(np.array([0.6544417*np.ones_like(X0.reshape(-1)),0.5664773*np.ones_like(X0.reshape(-1)), X0.reshape(-1)/100,Y0.reshape(-1)/100,Z0.reshape(-1)/100])).T
# Xin=Xin.float().to(device)
# Y,_=model_traveltime(Xin)
# VV=Y.cpu()[:,0].reshape(26,26,26).detach().clone().numpy()
# (VV-TT1).min(),(VV-TT1).max()

In [23]:
travel_time_list_test=get_traveling_time_multi([V_test],station_df,config,dx=dx)
pick_df = []
for source_index in range(n_source):
    for station_index in range(n_station):
        
        time=travel_time_list_test[0,station_index,source_loc_lindex[source_index][0],source_loc_lindex[source_index][1],source_loc_lindex[source_index][2]]
        time=time+source_time[source_index] 
        pick_df.append({
            "id": 'S.'+str(station_index)+'.BH',
            "timestamp": np.datetime64(int(1000*time), 'ms'),
            #"timestamp": np.datetime64(int(1000*time_true), 'ms'),
            "prob": 1.0,
            "type": 'P',
            "true_ed": source_index
        })
pick_df = pd.DataFrame(pick_df)

100%|██████████| 1/1 [00:00<00:00,  2.59it/s]


In [ ]:

config={}
config['x(km)']=[0,100]
config['y(km)']=[0,100]
config['z(km)']=[0,100]
config["center"] = ((config['x(km)'][0]+config['x(km)'][1])/2, (config['y(km)'][0]+config['y(km)'][1])/2)
config['neural_field']=True
config['optimize_wave_speed']=True
config['optimize_wave_speed_after_decay']=False
config['DBSCAN']=False
config['beta_z']=0.5
config['wave_speed_model_hidden_dim']=2
config['P_phase']=True
config['S_phase']=False
config["vel"] = {"P": v_background, "S": 2.0}
config["time_before"]=5
config["min_peak_pre_event"]=13
# config['init'] = 'test'
# config['true_xyz']=source_loc
# config['true_time']=source_time
# config['true_z']=[0.66429675,0.6150655]
# config['boundary_rate']=0
# config['lr']=0.000000000000001

config['noise'] = 5e-3
config['patience_max'] = 100



In [25]:
from harpa import association

picks,catalogs,z=association(pick_df,station_df,config,verbose=10,model_traveltime=model_traveltime.to('cpu'))


Associating 160 picks separated into 1 slides with 1 CPUs


  0%|          | 0/1 [00:00<?, ?it/s]

max number of event in this slice: 8 number of picks in this slice: 160
{'neural_field': True, 'optimize_wave_speed': True, 'optimize_wave_speed_after_decay': False, 'lr': 0.01, 'noise': 0.005, 'patience_max': 100, 'lr_decay': 0.1, 'epoch_before_decay': 10000, 'epoch_after_decay': 10000, 'dbscan_min_samples': 1, 'DBSCAN': False, 'max_time_residue': 2, 'min_peak_pre_event': 16, 'min_peak_pre_event_s': 0, 'min_peak_pre_event_p': 0, 'n_event_max_rate': 1, 'n_event_max': 8, 'second_adjust': True, 'P_phase': True, 'S_phase': False, 'remove_overlap_events': False, 'denoise_rate': 0, 'beta_time': 0.5, 'beta_space': 0.5, 'beta_z': 0.5, 'init': 'data', 'boundary_rate': 0.05, 'x(km)': (-5.0, 105.0), 'y(km)': (-5.0, 105.0), 'z(km)': (-5.0, 110.0), 'center': (50.0, 50.0), 'wave_speed_model_hidden_dim': 2, 'vel': {'P': 6, 'S': 2.0}, 'time_before': 5, 'ncpu': 1, 'start_time_ref': UTCDateTime(1970, 1, 1, 0, 0, 1, 988000), 't(s)': [-5.0, 175.896]}
Running SGLD...


/scicore/home/dokman0000/shi0000/phase_association/harpa/sgld.py:52: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  square_avg.mul_(alpha).addcmul_(1-alpha, d_p, d_p)


Step: 99, Loss: 22.83, z: [0.5393092  0.64974064]
Step: 199, Loss: 22.57, z: [0.5499171  0.65130454]
Step: 299, Loss: 22.24, z: [0.55828387 0.64681894]
Step: 399, Loss: 23.76, z: [0.5702641 0.6456688]
Step: 499, Loss: 22.68, z: [0.565794   0.63403016]
Step: 599, Loss: 21.75, z: [0.5746125 0.631763 ]
Step: 699, Loss: 22.38, z: [0.5772715 0.6273204]
Step: 799, Loss: 22.96, z: [0.5743092 0.6178858]
Step: 899, Loss: 23.62, z: [0.5814463  0.61057687]
Step: 999, Loss: 22.13, z: [0.5833737  0.60899884]
Step: 1099, Loss: 23.07, z: [0.58825934 0.59593755]
Step: 1199, Loss: 23.37, z: [0.5872582  0.58469653]
Step: 1299, Loss: 22.33, z: [0.5927301  0.58551556]
Step: 1399, Loss: 22.75, z: [0.5944284  0.58062935]
Step: 1499, Loss: 22.42, z: [0.6021102  0.57945424]
Step: 1599, Loss: 22.78, z: [0.60072637 0.5749673 ]
Step: 1699, Loss: 21.43, z: [0.6047118  0.56689644]
Step: 1799, Loss: 22.62, z: [0.60284376 0.55931795]
Step: 1899, Loss: 22.38, z: [0.6008132 0.5556183]
Step: 1999, Loss: 22.20, z: [0.60

100%|██████████| 1/1 [00:54<00:00, 54.08s/it]


Step: 19999, Loss: 21.61, z: [0.6488691  0.43625814]
Final SGLD loss: 20.01
Reindexing picks and events...


100%|██████████| 1/1 [00:00<00:00, 1037.42it/s]

removed 0 repeated events, left 0 unique events


ValueError: min() arg is an empty sequence

# Plot (optional, plotly required)

In [ ]:
from harpa.plt_utils import plot3D
import plotly.graph_objects as go

In [ ]:
from harpa.plt_utils import plot3D
import plotly.graph_objects as go
fig=plot3D(V_test,X0,vmin=1,vmax=18,colorbar=True,show=False)

fig.add_trace(go.Scatter3d(x=station_df['x(km)']/dx, y=station_df['y(km)']/dx,z=-station_df['z(km)']/dx, mode='markers',marker=dict(color='green',symbol='diamond',size=8)))

fig.add_trace(go.Scatter3d(x=catalog_df['x(km)']/dx, y=catalog_df['y(km)']/dx,z=-catalog_df['z(km)']/dx, mode='markers',marker=dict(color='black',symbol='circle',size=8)))

fig.add_trace(go.Scatter3d(x=catalogs['x(km)']/dx, y=catalogs['y(km)']/dx,z=-catalogs['z(km)']/dx, mode='markers',marker=dict(color='black',symbol='cross',size=8)))



In [32]:
catalog_df

,event_index,time,x(km),y(km),z(km)
0,0,1970-01-01T00:00:00.000000Z,72.0,80.0,0.0
1,1,1970-01-01T00:00:22.857000Z,80.0,80.0,0.0
2,2,1970-01-01T00:00:45.714000Z,60.0,32.0,0.0
3,3,1970-01-01T00:01:08.571000Z,4.0,8.0,0.0
4,4,1970-01-01T00:01:31.428000Z,40.0,4.0,0.0
5,5,1970-01-01T00:01:54.285000Z,44.0,84.0,0.0
6,6,1970-01-01T00:02:17.142000Z,96.0,16.0,0.0
7,7,1970-01-01T00:02:40.000000Z,48.0,76.0,0.0


In [ ]:

model_autoencoder=model_autoencoder.to('cpu')
z=torch.tensor(z)
beta=1
p=torch.logit(z/beta)
V_re=model_autoencoder.decoder(p.reshape(1,-1)).detach().clone().cpu().numpy()[0][0]
grid_size_x= int((config['x(km)'][1]-config['x(km)'][0])/dx+1)
grid_size_y= int((config['y(km)'][1]-config['y(km)'][0])/dx+1)
grid_size_z= int((config['z(km)'][1]-config['z(km)'][0])/dx+1)
x = np.linspace(config['x(km)'][0],config['x(km)'][1],grid_size_x)
y = np.linspace(config['y(km)'][0],config['y(km)'][1],grid_size_y)
z = np.linspace(config['z(km)'][0],config['z(km)'][1],grid_size_z)
X, Y, Z = np.meshgrid(x,y,z)
fig=plot3D(V_re,X,vmin=1,vmax=18,colorbar=True)

/tmp/ipykernel_10963/1447909517.py:2: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [ ]:
    # fig.add_trace(go.Scatter3d(x=loc_source[:,0]*(grid_size-1), y=loc_source[:,1]*(grid_size-1),z=-loc_source[:,2]*(grid_size-1), mode='markers',marker=dict(color='green',symbol='diamond',size=5)))

    # fig.add_trace(go.Scatter3d(x=loc_earthquake_coord[:,0]*(grid_size-1), y=loc_earthquake_coord[:,1]*(grid_size-1),z=-loc_earthquake_coord[:,2]*(grid_size-1), mode='markers',marker=dict(color='black',symbol='circle',size=5)))

    # fig.add_trace(go.Scatter3d(x=lowest_loss_loc_earthquake_re[:,0]*(grid_size-1), y=lowest_loss_loc_earthquake_re[:,1]*(grid_size-1),z=-lowest_loss_loc_earthquake_re[:,2]*(grid_size-1), mode='markers',marker=dict(color='black',symbol='cross',size=8)))